# Brake Pedal Experiments

## Imports

In [2]:
from pathlib import Path
from datetime import date
import os
from typing import List
from copy import deepcopy
from revcan.reverse_engineering.models.experiment import Experiment, Extern_Alphanumeric_Signal

## Create Necessary Experiment Files

### TODO: 
Set & check the following parameters:
- **Car Model File Name**: Specify the name of the car model file (`{model}_{vin}_{date-created}.json`).
- **Car Model Directory**: Check the directory path where the car model file is stored (Default: `../../data/car_metadata`).
- **Experiment Folder**: Set the path for the experiment folder where the results will be saved (Default: `../../data/experiments/{car}/{experiment}/{date-today}`).

In [ ]:
car_model_file_name = "model_vin_date-created.json" # TODO: Set correct car model file name
car_model_dir = "../../data/car_metadata" # TODO: Check directory of car model file
experiment_folder = "../../data/experiments/car/brake_pedal/" + date.today().strftime("%Y-%m-%d") # TODO: Set directory where experiment files should be saved to


car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)


brake_pedal_status = ["brake_pedal_activated", "brake_pedal_deactivated"]

#create experiments
for status in brake_pedal_status:

    experiment_file_path = os.path.join(experiment_folder, f"{status}.json")
    experiment_name = f"{status}"
    experiment_description = "brake pedal activation signal discovery."
    signal_selection ="all"
    print(f"Experiment file path: {experiment_file_path}")
    !python ../scripts_for_doip_new/04_create_experiment.py --car_model_file_path {car_model_file_path} --experiment_file_path '{experiment_file_path}' --experiment_name '{experiment_name}' --experiment_description '{experiment_description}' --signal_selection {signal_selection}


## [Optional]: Set Experiment Names manually

In [ ]:
car_model_file_name = "model_vin_date-created.json"
car_model_dir = "../../data/car_metadata"
experiment_folder = "../../data/experiments/car/brake_pedal/"
brake_pedal_status = ["brake_pedal_activated", "brake_pedal_deactivated"]

car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

## 1. Measurements

For each status 5 measurements will be taken.

### 1.1 Experiment 1: Brake pedal not activated

Before executing the next Cell: Make sure the brake lights are not activated 

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "brake_pedal_deactivated"+".json")}' --activate_logging true --num_samples 3

### 1.2 Experiment 2: Brake pedal activated

Before executing the next Cell: Activate the brake pedal for the whole time of the experiment

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "brake_pedal_activated"+".json")}' --activate_logging true --num_samples 3

## [Optional] Display Experiment Metadata

In [ ]:
# Display Experiment
for state in brake_pedal_status:
    experiment_file = os.path.join(experiment_folder, f"{state}.json")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"

## Load Experiments

Load all relevant experiments needed for this analysis

In [ ]:
experiment_files = {}
experiments = {}

for state in brake_pedal_status:
    experiment_files[state] = os.path.join(experiment_folder, f"{state}.json")
    experiments[state] = Experiment.load(experiment_files[state])

## Add Ground truth
Add ground truth data for steering wheel angle. By experiment design these are fixed values, and are added manually in this case

In [ ]:
for state in brake_pedal_status:
    experiments[state].external_alphanumeric_measurements.append(Extern_Alphanumeric_Signal( name = "Brake_Pedal_Status", id = 1, values = []))
    values:List[str] = []
    for value in experiments[state].measurements[0].values:
        values.append(state)
    experiments[state].external_alphanumeric_measurements[0].name = "Brake_Pedal_Status"
    experiments[state].external_alphanumeric_measurements[0].values = deepcopy(values)
    print(f"Experiment gear {state}: {len(experiments[state].external_alphanumeric_measurements[0].values)} ground truth values")

## Concat all experiments to combined_experiment

In [ ]:
combined_experiment_file_name = "experiment_non_constant_signals"
combined_experiment_file_path = os.path.join(experiment_folder, f"{combined_experiment_file_name}.json")
combined_experiment_name = f"Non constant signals for brake pedal activation experiment"
combined_experiment_description = f"Non constant signals for brake pedal activation experiment"
signal_selection ="all"
print(f"Experiment file path: {combined_experiment_file_path}")
!python ../scripts_for_doip_new/04_create_experiment.py --car_model_file_path {car_model_file_path} --experiment_file_path '{combined_experiment_file_path}' --experiment_name '{combined_experiment_name}' --experiment_description '{combined_experiment_description}' --signal_selection {signal_selection}

#experiment_file = os.path.join(experiment_folder, f"{combined_experiment_file_name}.json")
combined_experiment = Experiment.load(combined_experiment_file_path)
combined_experiment.external_alphanumeric_measurements.append(Extern_Alphanumeric_Signal( name = "Brake_Pedal_Status", id = 1, values = []))

for state in brake_pedal_status:
    combined_experiment.experiment_runtime_seconds += experiments[state].experiment_runtime_seconds
    for measurement in experiments[state].measurements:
            for x in combined_experiment.measurements:
                if x.serverid == measurement.serverid:
                    if x.did == measurement.did:
                        x.values.extend(measurement.values)
    combined_experiment.external_alphanumeric_measurements[0].values.extend(experiments[state].external_alphanumeric_measurements[0].values)


## Filter

Keep only non constant signals

In [ ]:
number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements before filtering", )
print("Applaying remove constant values filter")

combined_experiment.keep_non_constant_signals(combined_experiment)

number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements after filtering", )

## Save combined experiment file

In [ ]:
combined_experiment_file = os.path.join(experiment_folder, f"experiment_non_constant_signals.json")
combined_experiment.save(f"{combined_experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{combined_experiment_file}"

## [OPTIONAL] Remove all previously found constant signals from individual experiments

In [ ]:
for state in brake_pedal_status:
    experiments[state].keep_signals_by_list(signals_list=combined_experiment.measurements)
    experiment_file = os.path.join(experiment_folder, f"{state}.json")
    experiments[state].save(f"{experiment_file}")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"